<a href="https://colab.research.google.com/github/nunocesarsa/Examples/blob/main/SimpleOperations/PCA_on_a_Raster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#loading packages

In [49]:
!pip install pyrsgis
!pip install rasterio
!pip install pyproj
!pip install earthpy
!pip install rasterstats

from pyrsgis.convert import changeDimension
from pyrsgis import raster
import rasterio
import pyproj
import earthpy.spatial as es

import numpy as np
import pandas as pd

import os
from glob import glob

import sklearn
from sklearn import decomposition

import rasterstats

     |████████████████████████████████| 133kB 12.9MB/s 


#Downloading raster data from some guys tutorial 

In [6]:
!git clone https://github.com/urbanSpatial/classifying_satellite_imagery_in_R

Cloning into 'classifying_satellite_imagery_in_R'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 106 (delta 2), reused 23 (delta 2), pack-reused 80
Receiving objects: 100% (106/106), 42.79 MiB | 7.13 MiB/s, done.
Resolving deltas: 100% (30/30), done.


#loading and stacking the data

In [ ]:
#fetches the iamges at 30m excluding the panchromatic and cirrus 
landsat_bands_data_path = "/content/classifying_satellite_imagery_in_R/data/band*[1-7]*.tif"

stack_band_paths = glob(landsat_bands_data_path)
stack_band_paths.sort()

#resorting because of the band names 
stack_band_paths_sorted = [stack_band_paths[i] for i in [0,3,4,5,6,7,8]]

# Create image stack and apply nodata value for Landsat
arr_st, meta = es.stack(stack_band_paths_sorted, nodata=-9999)


#and saving to a file in outputs folder
!mkdir outputs
es.stack(stack_band_paths_sorted, out_path='/content/outputs/Landsat.tif') #fails because of uint16 if we add the thermals


#loading the multiband stack

In [50]:
#lading the landsat data 
ds1, bands = raster.read('/content/outputs/Landsat.tif')

print(ds1)
print('Original data:',bands.shape)

bandByPixel = changeDimension(bands) #we have to devide all values by 10k - its a conversion from bits to reflectances
bandByPixel_t = np.transpose(bandByPixel)

print('Converted:',bandByPixel.shape)
print('Converted & transposed:',bandByPixel_t.shape)

Original data: (7, 1413, 1121)
Converted: (1583973, 7)
Converted & transposed: (7, 1583973)


#Calculating the PCA

In [35]:
pca = sklearn.decomposition.PCA(bandByPixel_t)
pca.n_components.shape
np.transpose(pca.n_components).shape

(1583973, 7)

In [62]:
#Reconverting
out_pca_t = pca.n_components

out_pca = np.transpose(out_pca_t)

print('PCA_t',out_pca_t.shape)
print('PCA',out_pca.shape)

PCA_t (7, 1583973)
PCA (1583973, 7)


#Saving the raster

In [70]:
#this saves all the components into separate bands
for i in range(out_pca.shape[1]):
  print("Saving component:",i+1)

  pca_i = np.reshape(out_pca[:,i],(ds1.RasterYSize,ds1.RasterXSize))
  out_name = '/content/outputs/LandsatPCA_'+str(i+1)+'.tif'
  raster.export(pca_i,ds1,out_name,dtype='float')


Saving component: 1
Saving component: 2
Saving component: 3
Saving component: 4
Saving component: 5
Saving component: 6
Saving component: 7


#Creating a multiband raster and saving

In [85]:
#fetches the iamges at 30m excluding the panchromatic and cirrus 
landsat_bands_data_path = "/content/outputs/LandsatPCA_[1-7]*.tif"
stack_band_paths = glob(landsat_bands_data_path)

stack_band_paths #comment from here in case it doesn't need sorting
stack_band_paths.sort() 
stack_band_paths

['/content/outputs/LandsatPCA_1.tif',
 '/content/outputs/LandsatPCA_2.tif',
 '/content/outputs/LandsatPCA_3.tif',
 '/content/outputs/LandsatPCA_4.tif',
 '/content/outputs/LandsatPCA_5.tif',
 '/content/outputs/LandsatPCA_6.tif',
 '/content/outputs/LandsatPCA_7.tif']

#Saving the multiband PCA

In [ ]:
es.stack(stack_band_paths, out_path='/content/outputs/Landsat_PCA.tif') #fails because of uint16 if we add the thermals
